In [1]:
import os
import numpy as np
from pathlib import Path
import mne
from mne import create_info
from mne.io import RawArray
from mne_bids import BIDSPath, get_entity_vals, read_raw_bids
from eztrack.fragility import DiscreteLinearSystem
from eztrack.fragility.fragility import _reference_raw, state_lds_derivative
from eztrack.io.array import DerivativeArray
from eztrack.io.deriv_info import create_deriv_info

In [2]:
bids_root = "C:/Users/patri/OneDrive - Johns Hopkins/scalp_reconstruction/unfiltered"
sourcedata = Path(bids_root) / "sourcedata"
deriv_dir = Path(bids_root) / "derivatives"

In [3]:
session = 'monitor'
task = 'monitor'
run = "01"
datatype = "eeg"
montage = "standard_1020"
ext = ".edf"
reference="monopolar"

In [4]:
subjects = get_entity_vals(bids_root, 'subject')
bids_fpaths = [BIDSPath(subject=s, session=session, task=task, run=run, datatype=datatype, extension=ext, root=bids_root).match() for s in subjects]
bids_fpaths = [item for sublist in bids_fpaths for item in sublist]

In [5]:
keep_chs = ['Fp1', 'Fp2', 'F3','F4', 'F7', 'F8', 'Fz','T3', 'T4', 'C3', 'C4', 'Cz', 'T5', 'T6', 'P3', 'P4', 'Pz', 'O1', 'O2', 'T7', 'T8', 'P7', 'P8']

In [6]:
def get_channel_map(raw_chs, rename_chs):
    ch_map = {}
    for ch in rename_chs:
        for rch in raw_chs:
            rch_base = rch.split("-")[0]
            if ch.upper() == rch_base.upper():
                ch_map[rch] = ch
    return ch_map

In [7]:
ltv_kwargs = {
    "winsize": 125,
    "stepsize": 125,
    "l2penalty": 0,
    "method_to_use": "pinv",
}

In [8]:
def num_steps(sfreq, winsize):
    return (sfreq*winsize) / 1000

In [9]:
#bids_fpaths = [bids_fpaths[0]]
for fpath in bids_fpaths:
    raw = read_raw_bids(fpath)
    raw.pick_types(eeg=True)
    rename_map = get_channel_map(raw.ch_names, keep_chs)
    raw.rename_channels(get_channel_map(raw.ch_names, keep_chs))
    raw.pick_channels(keep_chs)
    A_mats = state_lds_derivative(raw, **ltv_kwargs)
    sfreq = raw.info["sfreq"]
    data = raw.get_data()
    data_reconstruct = []
    for win in range(A_mats.shape[2]):
        A = A_mats.get_data()[:, :, win]
        step = ltv_kwargs['stepsize']
        x0 = data[:, win*step]
        A_mat = DiscreteLinearSystem(A)
        x_reconstruct = np.array(A_mat.reconstruct(x0, ltv_kwargs['winsize']-1))
        if len(data_reconstruct)>0:
            data_reconstruct = np.hstack((data_reconstruct,x_reconstruct))
        else:
            data_reconstruct = x_reconstruct
    ch_types = raw.get_channel_types()
    deriv_info = create_info(
        ch_names=raw.ch_names,
        sfreq=sfreq,
        ch_types=ch_types,
    )
    reconstructed = RawArray(data_reconstruct, info=deriv_info, first_samp=0, copy="auto", verbose=False)
    reconstructed_base = "C:/Users/patri/OneDrive - Johns Hopkins/scalp_reconstruction/unfiltered/derivatives/reconstructed"
    reconstructed_fpath = fpath.update(root=reconstructed_base, extension=".fif")
    Path(reconstructed_fpath.directory).mkdir(parents=True, exist_ok=True)
    reconstructed.save(reconstructed_fpath, overwrite=True)

<ipython-input-9-0d20a7f05937>:3: RuntimeWarning: Did not find any events.tsv associated with sub-00000355_ses-monitor_task-monitor_run-01.

The search_str was "C:\Users\patri\OneDrive - Johns Hopkins\scalp_reconstruction\unfiltered\sub-00000355\**\sub-00000355_ses-monitor*events.tsv"
  raw = read_raw_bids(fpath)
100%|██████████| 2503/2503 [00:05<00:00, 482.61it/s]
--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\patri\AppData\Local\Programs\Python\Python38\lib\logging\handlers.py", line 70, in emit
    self.doRollover()
  File "C:\Users\patri\AppData\Local\Programs\Python\Python38\lib\logging\handlers.py", line 171, in doRollover
    self.rotate(self.baseFilename, dfn)
  File "C:\Users\patri\AppData\Local\Programs\Python\Python38\lib\logging\handlers.py", line 111, in rotate
    os.rename(source, dest)
PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'd:\\desktop\\eztrack\\.eztrack\\logging\\eztrack

<ipython-input-9-0d20a7f05937>:32: RuntimeWarning: This filename (C:\Users\patri\OneDrive - Johns Hopkins\scalp_reconstruction\unfiltered\derivatives\reconstructed\sub-00000355\ses-monitor\eeg\sub-00000355_ses-monitor_task-monitor_run-01_eeg.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  reconstructed.save(reconstructed_fpath, overwrite=True)
<ipython-input-9-0d20a7f05937>:3: RuntimeWarning: Did not find any events.tsv associated with sub-00000592_ses-monitor_task-monitor_run-01.

The search_str was "C:\Users\patri\OneDrive - Johns Hopkins\scalp_reconstruction\unfiltered\sub-00000592\**\sub-00000592_ses-monitor*events.tsv"
  raw = read_raw_bids(fpath)
100%|██████████| 2885/2885 [00:00<00:00, 12706.37it/s]
--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\patri\AppData\Local\Programs\Python\Python38\lib\logging\handlers.py", line 70, in em

<ipython-input-9-0d20a7f05937>:32: RuntimeWarning: This filename (C:\Users\patri\OneDrive - Johns Hopkins\scalp_reconstruction\unfiltered\derivatives\reconstructed\sub-00000592\ses-monitor\eeg\sub-00000592_ses-monitor_task-monitor_run-01_eeg.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  reconstructed.save(reconstructed_fpath, overwrite=True)
<ipython-input-9-0d20a7f05937>:3: RuntimeWarning: Did not find any events.tsv associated with sub-00000767_ses-monitor_task-monitor_run-01.

The search_str was "C:\Users\patri\OneDrive - Johns Hopkins\scalp_reconstruction\unfiltered\sub-00000767\**\sub-00000767_ses-monitor*events.tsv"
  raw = read_raw_bids(fpath)
100%|██████████| 3313/3313 [00:00<00:00, 11077.78it/s]
--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\patri\AppData\Local\Programs\Python\Python38\lib\logging\handlers.py", line 70, in em

<ipython-input-9-0d20a7f05937>:32: RuntimeWarning: This filename (C:\Users\patri\OneDrive - Johns Hopkins\scalp_reconstruction\unfiltered\derivatives\reconstructed\sub-00000767\ses-monitor\eeg\sub-00000767_ses-monitor_task-monitor_run-01_eeg.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  reconstructed.save(reconstructed_fpath, overwrite=True)
<ipython-input-9-0d20a7f05937>:3: RuntimeWarning: Did not find any events.tsv associated with sub-00000883_ses-monitor_task-monitor_run-01.

The search_str was "C:\Users\patri\OneDrive - Johns Hopkins\scalp_reconstruction\unfiltered\sub-00000883\**\sub-00000883_ses-monitor*events.tsv"
  raw = read_raw_bids(fpath)
100%|██████████| 2387/2387 [00:00<00:00, 9820.82it/s] 
--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\patri\AppData\Local\Programs\Python\Python38\lib\logging\handlers.py", line 70, in em

<ipython-input-9-0d20a7f05937>:32: RuntimeWarning: This filename (C:\Users\patri\OneDrive - Johns Hopkins\scalp_reconstruction\unfiltered\derivatives\reconstructed\sub-00000883\ses-monitor\eeg\sub-00000883_ses-monitor_task-monitor_run-01_eeg.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  reconstructed.save(reconstructed_fpath, overwrite=True)
<ipython-input-9-0d20a7f05937>:3: RuntimeWarning: Did not find any events.tsv associated with sub-00000930_ses-monitor_task-monitor_run-01.

The search_str was "C:\Users\patri\OneDrive - Johns Hopkins\scalp_reconstruction\unfiltered\sub-00000930\**\sub-00000930_ses-monitor*events.tsv"
  raw = read_raw_bids(fpath)
100%|██████████| 2599/2599 [00:00<00:00, 11346.78it/s]
--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\patri\AppData\Local\Programs\Python\Python38\lib\logging\handlers.py", line 70, in em

...

<ipython-input-9-0d20a7f05937>:32: RuntimeWarning: This filename (C:\Users\patri\OneDrive - Johns Hopkins\scalp_reconstruction\unfiltered\derivatives\reconstructed\sub-5\ses-monitor\eeg\sub-5_ses-monitor_task-monitor_run-01_eeg.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  reconstructed.save(reconstructed_fpath, overwrite=True)
<ipython-input-9-0d20a7f05937>:3: RuntimeWarning: Scaling factor is not defined in following channels:
POL SpO2, POL EtCO2, POL Pulse, POL CO2Wave
  raw = read_raw_bids(fpath)
<ipython-input-9-0d20a7f05937>:3: RuntimeWarning: Physical range is not defined in following channels:
POL SpO2, POL EtCO2, POL Pulse, POL CO2Wave
  raw = read_raw_bids(fpath)
100%|██████████| 1520/1520 [00:00<00:00, 8349.73it/s]
--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\patri\AppData\Local\Programs\Python\Python38\lib\logging\handle

<ipython-input-9-0d20a7f05937>:32: RuntimeWarning: This filename (C:\Users\patri\OneDrive - Johns Hopkins\scalp_reconstruction\unfiltered\derivatives\reconstructed\sub-6\ses-monitor\eeg\sub-6_ses-monitor_task-monitor_run-01_eeg.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  reconstructed.save(reconstructed_fpath, overwrite=True)
<ipython-input-9-0d20a7f05937>:3: RuntimeWarning: Scaling factor is not defined in following channels:
POL SpO2, POL EtCO2, POL Pulse, POL CO2Wave
  raw = read_raw_bids(fpath)
<ipython-input-9-0d20a7f05937>:3: RuntimeWarning: Physical range is not defined in following channels:
POL SpO2, POL EtCO2, POL Pulse, POL CO2Wave
  raw = read_raw_bids(fpath)
100%|██████████| 2080/2080 [00:00<00:00, 11002.80it/s]
--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\patri\AppData\Local\Programs\Python\Python38\lib\logging\handl

<ipython-input-9-0d20a7f05937>:32: RuntimeWarning: This filename (C:\Users\patri\OneDrive - Johns Hopkins\scalp_reconstruction\unfiltered\derivatives\reconstructed\sub-7\ses-monitor\eeg\sub-7_ses-monitor_task-monitor_run-01_eeg.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  reconstructed.save(reconstructed_fpath, overwrite=True)
<ipython-input-9-0d20a7f05937>:3: RuntimeWarning: Scaling factor is not defined in following channels:
POL SpO2, POL EtCO2, POL Pulse, POL CO2Wave
  raw = read_raw_bids(fpath)
<ipython-input-9-0d20a7f05937>:3: RuntimeWarning: Physical range is not defined in following channels:
POL SpO2, POL EtCO2, POL Pulse, POL CO2Wave
  raw = read_raw_bids(fpath)
100%|██████████| 1568/1568 [00:00<00:00, 8207.63it/s]
--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\patri\AppData\Local\Programs\Python\Python38\lib\logging\handle

<ipython-input-9-0d20a7f05937>:32: RuntimeWarning: This filename (C:\Users\patri\OneDrive - Johns Hopkins\scalp_reconstruction\unfiltered\derivatives\reconstructed\sub-8\ses-monitor\eeg\sub-8_ses-monitor_task-monitor_run-01_eeg.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  reconstructed.save(reconstructed_fpath, overwrite=True)
<ipython-input-9-0d20a7f05937>:3: RuntimeWarning: Scaling factor is not defined in following channels:
POL SpO2, POL EtCO2, POL Pulse, POL CO2Wave
  raw = read_raw_bids(fpath)
<ipython-input-9-0d20a7f05937>:3: RuntimeWarning: Physical range is not defined in following channels:
POL SpO2, POL EtCO2, POL Pulse, POL CO2Wave
  raw = read_raw_bids(fpath)
100%|██████████| 1615/1615 [00:00<00:00, 8365.98it/s]
--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\patri\AppData\Local\Programs\Python\Python38\lib\logging\handle

<ipython-input-9-0d20a7f05937>:32: RuntimeWarning: This filename (C:\Users\patri\OneDrive - Johns Hopkins\scalp_reconstruction\unfiltered\derivatives\reconstructed\sub-9\ses-monitor\eeg\sub-9_ses-monitor_task-monitor_run-01_eeg.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  reconstructed.save(reconstructed_fpath, overwrite=True)


In [ ]:
reconstructed_fpath = fpath.update(root=reconstructed_base, extension=".fif")
Path(reconstructed_fpath.directory).mkdir(parents=True, exist_ok=True)
reconstructed.save(reconstructed_fpath, overwrite=True)
reconstructed_fpath

In [ ]:
raw = mne.io.Raw("C:/Users/patri/OneDrive - Johns Hopkins/scalp_reconstruction/unfiltered/derivatives/reconstructed/sub-00000355/ses-monitor/eeg/sub-00000355_ses-monitor_task-monitor_run-01_eeg.fif")

In [ ]:
raw.info